# Fetch books using google api

In [14]:
import requests
import json
import os

In [15]:
api = 'https://www.googleapis.com/books/v1/volumes'
base_folder = './topics'
os.listdir(base_folder)

['Computers & Technology',
 'Business & Investing',
 'Art, Music & Photography',
 'Science & Math',
 'Literature & Fiction',
 'Comics & Graphics Novels']

In [16]:
folder = base_folder + '/Computers & Technology'
catg = ['Apple', 'CAD', 'Certification', 'Computer Science', 'Databases', 'Desktop Publishing', 'E-Commerce', 'Graphic Design', 'Internet & Social Media', 'Microsoft', 'Networking', 'Operating System', 'Phones & Tablets', 'Programming', 'Software', 'Video Games', 'Web Design', 'Web Development']

In [17]:
def fetch(subject, file, q=0, folder=folder, loop=3):
    arr = []
    api = 'https://www.googleapis.com/books/v1/volumes'
    for l in range(loop):
        if not q:
            url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=' + str(l * 40) + '&maxResults=40'
        else:
            url = api + '?q=' + subject + '&langRestrict=en&startIndex=' + str(l * 40) + '&maxResults=40'
        res = requests.get(url).json()
        if not res['totalItems']:
            break
        arr += res['items']
        if len(res['items']) < 40:
            break
    if len(arr):
        print(len(arr))
        with open(folder + '/' + file + '.json', 'w') as f:
            json.dump(arr, f)

In [18]:
file = catg[17]
print(file)
subject = file + '+computers'

Web Development


In [19]:
# url = 'https://www.googleapis.com/books/v1/volumes?q=subject:' +subject + '&langRestrict=en&maxResults=40'
url = 'https://www.googleapis.com/books/v1/volumes?q=' +subject + '&langRestrict=en&maxResults=40'
res = requests.get(url).json()
print(res['totalItems'])
if res['totalItems']:
    res = res['items']
    for i in range(0, min(40, len(res))):
        print('title:' + res[i]['volumeInfo']['title'])
        if 'categories' in res[i]['volumeInfo']:
            print('category: ' + str(res[i]['volumeInfo']['categories']))
        if 'description' in res[i]['volumeInfo']:
            print('description: ' + res[i]['volumeInfo']['description'][:200])
        print()

3284
title:Multimedia Web Design and Development
category: ['Computers']
description: This book/DVD package introduces the necessary steps and stages of planning a modern multimedia Web site. It includes both the design and development aspects for novices and a complete plan to get you

title:Human Computer Interaction Research in Web Design and Evaluation
category: ['Computers']
description: "This is a comprehensive book on Human Computer Interaction and Web design focusing on various areas of research including theories, analysis, design and evaluation. It is not a book on web programmin

title:PHP and MySQL Web Development
category: ['Computers']
description: Explains how to access and create MySQL databases through PHP scripting, including authentication, network connectivity, session management, and content customization.

title:Node.js Web Development
category: ['Computers']
description: Create real-time server-side applications with this practical, step-by-step guide About This 

## Fetch Function

In [196]:
fetch(subject, file, q=0, loop=3)

120


## Fetching Manually

In [186]:
arr = []
url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=0&maxResults=40'
# url = api + '?q=' + subject + '&langRestrict=en&startIndex=0&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

17

In [189]:
# url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=40&maxResults=40'
url = api + '?q=' + subject + '&langRestrict=en&startIndex=40&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

97

In [116]:
# url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=80&maxResults=40'
url = api + '?q=' + subject + '&langRestrict=en&startIndex=80&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

120

In [190]:
with open(folder + '/' + file + '.json', 'w') as f:
    json.dump(arr, f)

# Json to CSV

In [7]:
import os
import json
import csv

base_folder = './topics'
book_list = []

label = 0

for folder in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        with open(file_path) as f:
            books = json.load(f)
            for book in books:
                book = book['volumeInfo']
                title = book['title'] if 'title' in book else ""
                description = book['description'] if 'description' in book else ""
                if 'authors' in book:
                    authors = ''
                    for author in book['authors']:
                        authors += author + ", "
                    authors = authors.strip(', ')
                else:
                    authors = ''
                text = title + " " + authors + " " + description
                text = text.strip(' ')
                book_list.append({'label': label, 'category': folder, 'subcategory': file.split('.')[0], 'text': text })
        label += 1

with open('book_dataset.csv', 'w') as f:
    w = csv.DictWriter(f, book_list[0].keys())
    w.writeheader()
    w.writerows(book_list)